# NeuralTrack

Project to explore the idea of tracking power, time, energy of a model as part of a larger comprehensive metric. Inspired by Neuralpower.

In [1]:
# pytorch version
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from model import FFNN

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [3]:
BATCH_SIZE = 64

train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

model = FFNN().to(device)
print(f"device: {device}")
print(model)

cuda
device: cuda
FFNN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=1024, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1024, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [5]:
def train(dataloader: DataLoader, model: FFNN, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1) * len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

def test(dataloader:DataLoader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f'test error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')

In [6]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-----------------------------------")
    train(train_dataloader, model, loss_fn=loss_fn, optimizer=optimizer)
    test(test_dataloader, model=model, loss_fn=loss_fn)

print('done')

Epoch 1
-----------------------------------
loss: 2.301170 [   64/60000]
loss: 2.282586 [ 6464/60000]
loss: 2.257360 [12864/60000]
loss: 2.251159 [19264/60000]
loss: 2.233510 [25664/60000]
loss: 2.183552 [32064/60000]
loss: 2.194309 [38464/60000]
loss: 2.144708 [44864/60000]
loss: 2.142738 [51264/60000]
loss: 2.101781 [57664/60000]
test error: 
 Accuracy: 52.2%, Avg loss: 2.089904 

Epoch 2
-----------------------------------
loss: 2.110566 [   64/60000]
loss: 2.087301 [ 6464/60000]
loss: 2.013222 [12864/60000]
loss: 2.033979 [19264/60000]
loss: 1.965706 [25664/60000]
loss: 1.896781 [32064/60000]
loss: 1.913766 [38464/60000]
loss: 1.820781 [44864/60000]
loss: 1.832439 [51264/60000]
loss: 1.737381 [57664/60000]
test error: 
 Accuracy: 61.6%, Avg loss: 1.738437 

Epoch 3
-----------------------------------
loss: 1.790141 [   64/60000]
loss: 1.743969 [ 6464/60000]
loss: 1.611377 [12864/60000]
loss: 1.662791 [19264/60000]
loss: 1.540902 [25664/60000]
loss: 1.506825 [32064/60000]
loss: 1.51

In [7]:
torch.save(model.state_dict(), "model.pth")
print("saved")

saved
